# Herramientas de apoyo para el análisis y procesado de datos del secuenciador de ADN MinION

**Autor:** Orlandy Ariel Sánchez Acosta.

**Correo:** alu0100773408@ull.edu.es

**Asignatura:** Trabajo de Fin de Grado.

** Este cuaderno se centrará en realizar uno de los filtros utilizados, en este caso el filtrado por contenido en GC.**

In [ ]:
FICHERO_ENTRADA = "DatosOriginales/secuencias_originales.fastq" #FICHERO original
FORMATO = "fastq"

In [ ]:
D_PORC_ATIPICOS = 5 #Esta variable será el porcentaje que se quiere quitar de datos atípicos
IT_CONFIANZA = 0.95 # Esta variable será la utilizada para calcular el intervalo de confianza.

In [ ]:
FICHERO_SALIDA_MALAS = "Lecturas_malas.fastq"
FICHERO_SALIDA_BUENAS = "Lecturas_buenas.fastq"

In [ ]:
PATH_SALIDA_GC = "Salidas/ContenidoGC/"

## Librerías 
En caso de dudas o cualquier consulta sobre las librerías usadas, consultar los siguientes enlaces.
* [Biopython](http://biopython.org/wiki/Documentation)
* [Plotly](https://plot.ly/python/)
* [Iteratools](https://docs.python.org/2/library/itertools.html)
* [IPython](https://ipython.readthedocs.io/en/stable/install/install.html)
* [Pandas](http://pandas.pydata.org/)
* [NumPy](http://www.numpy.org/)

A continuación, se pasará a importar todas las librerías/módulos que se utilizarán.

In [5]:
### Librería de BioInformática.
from Bio import SeqIO
from Bio.SeqUtils import GC
### Librería para realizar los gráficos 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
### Librerías de utilidades
import numpy as np
from IPython.display import Markdown,display, HTML
from scipy.stats import norm
### Librerías propias
import GraficoDePuntos as gp
import Utilidades as utile
import UtilidadesParaArrays as arrUtile
import PresentacionDatos as psd

In [6]:
IT_CONFIANZA = norm.ppf(IT_CONFIANZA)

In [7]:
c_externa_utile = utile.Utilidades() #llamada a una clase propia para realizar el porcentaje y las permutaciones.
c_arrays_utile = arrUtile.UtilidadesParaArrays()
c_psd = psd.PresentacionDatos()

## Lecturas de secuencias desde el fichero de entrada
Carga las lecturas del fichero en un array, de manera que solo lea una vez del fichero.

In [8]:
fichero_de_secuencias = [rec for rec in SeqIO.parse(FICHERO_ENTRADA, FORMATO)]

In [9]:
lecturas = [len(rec) for rec in fichero_de_secuencias]

In [10]:
tamanio = len(lecturas)#total de lecturas del fichero
minimo = min(lecturas)#lectrua más pequeña
maximo = max(lecturas)#lectura más grande
c_psd.printmd("* **El fichero: **%s"%FICHERO_ENTRADA)
c_psd.printmd("* **Total de READS (lecturas):** %i"%tamanio)
c_psd.printmd("* **Los READS(lecturas) tiene un tamaño de entre: **%i y %i"%(minimo,maximo))

* **El fichero: **DatosOriginales/secuencias_originales.fastq

* **Total de READS (lecturas):** 1171

* **Los READS(lecturas) tiene un tamaño de entre: **47 y 185942

# Contenido GC

Contenido GC o Porcentaje GC (contenido de guanina y citosina) es una característica del genoma de un organismo o de cualquier pedazo de ADN o ARN. G y C denotan guanina y citosina, respectivamente. Expresado generalmente como porcentaje, representa la cantidad de pares Guanina-Citosina en la molécula de ADN o genoma que está siendo investigado. La fracción restante de cualquier molécula de ADN contendrá bases A (adenina) y T (timina), de forma que el contenido GC da también el contenido AT.
[Leer más](https://es.wikipedia.org/wiki/Contenido_GC)

In [11]:
dic_contenido_gc = {} # Diccionario en donde se guarda el contenido gc, de cada secuencia, calculado
dic_gc_buenos = {} # diccionario que guarda solo el contenido gc de las secuencias que no tienen error
dic_gc_malos = {} # diccionario que guarda solo el contenido gc de las secuencias que tienen error

In [12]:
for i, secuencia in enumerate(fichero_de_secuencias):
    #contenido en gc
    dic_contenido_gc[i] = (GC(secuencia.seq))

**array_gc_valores** guarda el _contenido GC_ ordenados de menor a mayor.

In [13]:
array_gc_valores = sorted(GC(rec.seq) for rec in fichero_de_secuencias)

**sec_a_quitar ** se guarda el valor obtenido de porcentaje _ inverso() que no es más que el número de secuencias que se va a quitar para la media recortada.

In [14]:
sec_a_quitar = c_externa_utile.porcentaje_inverso(D_PORC_ATIPICOS,tamanio)

**inicio** y **fin** es la posición de inicio y final que tendrá el array ->**array_gc_val_recortados** creado a partir de **array_gc_valores**

In [15]:
inicio = sec_a_quitar
fin = (tamanio-inicio)

In [16]:
array_gc_val_recortados = array_gc_valores[inicio:fin]

**med_recortada_gc** y **dt_recortada_gc** son, respectivamente, la media y la desviación típica o estándar del array **array_gc_val_recortados**

In [17]:
med_recortada_gc = np.mean(array_gc_val_recortados)
dt_recortada_gc = np.std(array_gc_val_recortados)

Se preparan los diccionarios **dic_gc_buenos** y **dic_gc_malos** en los que se guardarán el contenido gc y la posición que pasen el filtro **[ nt ± IT_CONFIANZA * σ ] ** y los que no respectivamente.
>**Nota:** nt = media recortada

In [18]:
limite_superior = med_recortada_gc + IT_CONFIANZA * dt_recortada_gc
limite_inferior = med_recortada_gc - IT_CONFIANZA * dt_recortada_gc

In [19]:
for clave, valor in dic_contenido_gc.items():
    if(valor >= limite_inferior
      and valor <= limite_superior):
        dic_gc_buenos[clave] = valor
    else:
        dic_gc_malos[clave] = valor

In [20]:
titulo = "Lectura de secuencias Contenido GC"
leyenda_b = "% Contenido GC bueno"
leyenda_m = "% Contenido GC malo"

In [21]:
porc_buenos_gc = c_externa_utile.tanto_por_ciento(len(dic_gc_buenos),tamanio)

In [22]:
titulo_nombre_fich = FICHERO_ENTRADA.split("/")[1] #extraigo el nombre del fichero

In [23]:
titulo_graf ="<b> %s </b></br></br> <b>Fichero:</b> %s </br> %.2f%s </br>%.2f%s"%(titulo,
                                                             titulo_nombre_fich,
                                                             porc_buenos_gc,
                                                             leyenda_b,
                                                             (100-porc_buenos_gc),
                                                             leyenda_m)

In [24]:
grfPto=gp.GraficoDePuntos()
grafico = grfPto.crear_grafico_puntos(
                                    dic_gc_buenos,
                                    dic_gc_malos,
                                    titulo_graf,
                                    'Posición de la secuencia',
                                    "Contenido GC",
                                    tamanio,
                                    med_recortada_gc,
                                    IT_CONFIANZA,
                                    limite_inferior,
                                    limite_superior
                                   )
eliminar_btn_barra  = grfPto.eliminar_btn_barra()
plot(grafico,
     filename = 'SalidasGraficas/GraficoDePuntosGC.html',
     auto_open=False,
     config= eliminar_btn_barra
    )

'file://C:\\Users\\orlan\\Desktop\\TFG\\SalidasGraficas\\GraficoDePuntosGC.html'

In [25]:
HTML('<iframe src=SalidasGraficas/GraficoDePuntosGC.html width=100% height=550></iframe>')

# Visualizar las secuencias que se detectan como error en el contenido GC

Para ver las secuencias y comprobar que corresponde con los gráficos, se muestran los datos indicando la posición de la lectura, la secuencia y el contenido GC.

# Exportar resultados a ficheros FastQ
A continuación, se guardarán los resultados obtenidos en la siguiente jerarquía de directorios:
* **Método de Contenido GC**
    * Salidas/ContenidoGC/

>**NOTA:** Dentro de cada uno de estos directorios se encontrará un fichero FastQ con las lecturas fallidas y otro con las lecturas buenas.

**1.Primero se extraen las posiciones de las lecturas, tanto buenas como fallidas, de los tres métodos.**

In [26]:
a_gc_malos = c_arrays_utile.obtener_claves_dic(dic_gc_malos)
a_gc_buenos = c_arrays_utile.obtener_claves_dic(dic_gc_buenos)

**2.Se extraen las secuencias completas que se encuentran en las posiciones extraídas anteriormente.**

In [27]:
a_sec_gc_malos = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_gc_malos)
a_sec_gc_buenos = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_gc_buenos)

**3.Se procede a guardar las secuencias en los distintos ficheros de salidas.**

In [28]:
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_GC+FICHERO_SALIDA_MALAS,
                                    FORMATO,
                                    a_sec_gc_malos)
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_GC+FICHERO_SALIDA_BUENAS,
                                    FORMATO,
                                    a_sec_gc_buenos)